In [1]:
import requests
import json
from bs4 import BeautifulSoup
import urllib.request
import base64

In [2]:
import os
import olefile
import zlib
# pdf
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
import logging

### parameter
- seviceKey : id
- target : admrul
- query : 검색어(default: * \)
- numOfRows : 한 페이지 결과수
- pageNo : 페이지 번호

공공데이터 포털API로 행정규칙 id리스트를 얻기 위해 ServiceKey 발급
(https://www.data.go.kr/subMain.jsp?param=T1BFTkFQSUAxNTAwMDExNQ==#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTJtMSRAXnB1YmxpY0RhdGFQaz0xNTAwMDExNSRAXnB1YmxpY0RhdGFEZXRhaWxQaz11ZGRpOjI5ZDBkZjIyLWQxOWItNGY2Ny04Y2YyLWI3ZTM3MDQyZDc3Yl8yMDE4MDUxMTE0MzEkQF5vcHJ0aW5TZXFObz0yMDI4NCRAXm1haW5GbGFnPXRydWU=)

In [3]:
ServiceKey ='qHBjb9yik8lx%2B3bJ3tLeWux4VlZOlRcSe0otBb3Yh4T5c24Yia5vI6e%2BVFFUYHRwWl8FgTAvgRJjWf9XiIlpQg%3D%3D'

In [4]:
def get_url(page):
    id_ls = []
    url = 'http://apis.data.go.kr/1170000/law/admrulSearchList.do?'
    url += 'serviceKey=' + ServiceKey + '&query=*&target=admrul&numOfRows=10&pageNo='+ str(page)
    
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawl_ls = soup.select('admrul')

    for crawl in crawl_ls:
        crawl = str(crawl)

        start = crawl.index('ID=')
        crawl = crawl[start+3:]
        end = crawl.index('&')
        crawl = crawl[:end]

        id_ls.append(crawl)
    
    return id_ls

In [5]:
def get_file(id_ls):
    params = {}
    url_detail = 'http://www.law.go.kr/LSW/admRulAttFlList.do' 
    file_ls = []
    for admid in id_ls:
        file_info = []
        params.update({'admRulSeq':admid})
        res = requests.post(url_detail, data=params).text
        soup = BeautifulSoup(res,'html.parser')
        files = soup.select('ul li a')
        for file in files:
            file_info.append([file['href'],file.text])
        file_ls.append(file_info)
    
    return file_ls

In [6]:
def download(url, filename):
    if os.path.exists(filename) is False:
        with open(filename,"wb") as file:
            res = requests.get(url)
            file.write(res.content)

In [7]:
def read_pdf_file(pdfFile):
    logging.propagate = False 
    logging.getLogger().setLevel(logging.ERROR)
    with open(pdfFile, mode='rb') as file:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        process_pdf(rsrcmgr, device, file)
        device.close()

        content = retstr.getvalue()
        retstr.close()
    
    outputFile = pdfFile.split('.pdf')[0] + '.txt'
    
    with open(outputFile, mode='wt', encoding='utf-8') as output:
        output.write(content)

In [8]:
def read_hwp_file(hwpFile): 
    # PrvText부분은 압축되어 있지 않아 바로 보임
    with olefile.OleFileIO(hwpFile) as ole:
        f = ole.openstream('PrvText')
        encoded_text = f.read()
        decoded_text = encoded_text.decode('UTF-16')
        print(decoded_text)

    # 진짜 내용이 들어있는 BOdyText부분 - zlib 압축풀고 decoding
    with olefile.OleFileIO(hwpFile) as ole:
        print(ole.listdir())
        for section in ole.listdir():
            if 'BodyText' in section:
                f = ole.openstream(section)
                encoded_text = f.read()
                zobj = zlib.decompressobj(-zlib.MAX_WBITS)
                data = zobj.decompress(encoded_text)
                print(data)

In [9]:
url_file = 'http://www.law.go.kr/LSW/'
for page in range(1,10):
    id_ls = get_url(page)
    file_ls = get_file(id_ls)
    
    for files in file_ls:
        for file in files:
#             download(url_file+file[0],file[1])
            
            if 'hwp' in file[1]:
                pass
#                 read_hwp_file(file[1])
            elif 'pdf' in file[1]:
                download(url_file+file[0],file[1])
                read_pdf_file(file[1])
                break
            else:
                pass    # xls파일인 경우
            

KeyboardInterrupt: 